In [55]:
import sys
import os
import sqlite3 as sq
import pandas as pd
from pandas.io import sql
from datetime import datetime, timedelta
from pytz import timezone, all_timezones
from random import randint
import uuid

In [56]:
if sys.platform == 'linux': 
    Base=os.path.expanduser('~') + '/VKHCG'
else:
    Base='C:/VKHCG'

In [57]:
sInputFileName='C:/VKHCG/04-Clark/02-Assess/01-EDS/02-Python/Assess_People.csv'
sDataBaseDir='C:/VKHCG/04-Clark/03-Process/SQLite'
if not os.path.exists(sDataBaseDir):
    os.makedirs(sDataBaseDir)
sDatabaseName=sDataBaseDir + '/clark.db'
conn1 = sq.connect(sDatabaseName)
sDataVaultDir=Base + '/88-DV'
if not os.path.exists(sDataBaseDir):
    os.makedirs(sDataBaseDir)
sDatabaseName=sDataVaultDir + '/datavault.db'
conn2 = sq.connect(sDatabaseName)

In [58]:
sFileName=sInputFileName
print('################################')
print('Loading :',sFileName)
print('################################')
print(sFileName)
RawData=pd.read_csv(sFileName,header=0,low_memory=False, encoding="latin-1") 
print(RawData.shape)
RawData.head()

################################
Loading : C:/VKHCG/04-Clark/02-Assess/01-EDS/02-Python/Assess_People.csv
################################
C:/VKHCG/04-Clark/02-Assess/01-EDS/02-Python/Assess_People.csv
(20000, 4)


,FirstName,SecondName,LastName,Gender
0,Rheba,Darcie,Kong,Female
1,Norris,Solomon,Layne,Male
2,Alva,Fausto,Russo,Male
3,Barbara,Victoria,Dayton,Female
4,Antwan,Martin,Shin,Male


In [59]:
RawData.drop_duplicates(subset=None, keep='first', inplace=True)
start_date = datetime(1900,1,1,0,0,0)
start_date_utc=start_date.replace(tzinfo=timezone('UTC'))
HoursBirth=100*365*24
RawData['BirthDateUTC']=RawData.apply(lambda row:
 (start_date_utc + timedelta(hours=randint(0, HoursBirth)))
 ,axis=1)
zonemax=len(all_timezones)-1
RawData['TimeZone']=RawData.apply(lambda row:
 (all_timezones[randint(0, zonemax)])
 ,axis=1)
RawData['BirthDateISO']=RawData.apply(lambda row:
 row["BirthDateUTC"].astimezone(timezone(row['TimeZone']))
 ,axis=1)
RawData['BirthDateKey']=RawData.apply(lambda row:
 row["BirthDateUTC"].strftime("%Y-%m-%d %H:%M:%S")
 ,axis=1)
RawData['BirthDate']=RawData.apply(lambda row:
 row["BirthDateISO"].strftime("%Y-%m-%d %H:%M:%S")
 ,axis=1)
RawData['PersonID']=RawData.apply(lambda row:
 str(uuid.uuid4())
 ,axis=1)

In [60]:
Data=RawData.copy()
Data.drop('BirthDateUTC', axis=1,inplace=True) 
Data.drop('BirthDateISO', axis=1,inplace=True)
indexed_data = Data.set_index(['PersonID'])
print('################') 
sTable='Process_Person'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_data.to_sql(sTable, conn1, if_exists="replace")
print('################') 
PersonHubRaw=Data[['PersonID','FirstName','SecondName','LastName','BirthDateKey']]
PersonHubRaw['PersonHubID']=RawData.apply(lambda row:
 str(uuid.uuid4())
 ,axis=1)
PersonHub=PersonHubRaw.drop_duplicates(subset=None, \
 keep='first',\
inplace=False)
indexed_PersonHub = PersonHub.set_index(['PersonHubID'])
sTable = 'Hub-Person'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_PersonHub.to_sql(sTable, conn2, if_exists="replace")

################
Storing : C:/VKHCG/88-DV/datavault.db  Table: Process_Person
################
Storing : C:/VKHCG/88-DV/datavault.db  Table: Hub-Person


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [61]:
PersonSatelliteGenderRaw=Data[['PersonID','FirstName','SecondName','LastName'\
 ,'BirthDateKey','Gender']]
PersonSatelliteGenderRaw['PersonSatelliteID']=RawData.apply(lambda row:
 str(uuid.uuid4())
 ,axis=1)
PersonSatelliteGender=PersonSatelliteGenderRaw.drop_duplicates(subset=None, \
 keep='first', \
inplace=False)
indexed_PersonSatelliteGender = PersonSatelliteGender.set_index(['PersonSatelliteID'])
sTable = 'Satellite-Person-Gender'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_PersonSatelliteGender.to_sql(sTable, conn2, if_exists="replace") 

Storing : C:/VKHCG/88-DV/datavault.db  Table: Satellite-Person-Gender


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [62]:
PersonSatelliteBirthdayRaw=Data[['PersonID','FirstName','SecondName','LastName',\
 'BirthDateKey','TimeZone','BirthDate']]
PersonSatelliteBirthdayRaw['PersonSatelliteID']=RawData.apply(lambda row:
 str(uuid.uuid4())
 ,axis=1)
PersonSatelliteBirthday=PersonSatelliteBirthdayRaw.drop_duplicates(subset=None, \
 keep='first',\
 inplace=False)
indexed_PersonSatelliteBirthday = PersonSatelliteBirthday.set_index(['PersonSatelliteID'])
sTable = 'Satellite-Person-Names'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_PersonSatelliteBirthday.to_sql(sTable, conn2, if_exists="replace")

Storing : C:/VKHCG/88-DV/datavault.db  Table: Satellite-Person-Names


In [63]:
sFileDir='C:/VKHCG/04-Clark/03-Process/01-EDS/02-Python'
if not os.path.exists(sFileDir):
    os.makedirs(sFileDir)
    sOutputFileName = sTable + '.csv'
sFileName=sFileDir + '/' + sOutputFileName
print('################################')
print('Storing :', sFileName)
print('################################')
RawData.to_csv(sFileName, index = False)
print('Vacuum Databases')
sSQL="VACUUM;"
sql.execute(sSQL,conn1)
sql.execute(sSQL,conn2)
#################################################################
print('### Done!! ############################################')

################################
Storing : C:/VKHCG/04-Clark/03-Process/01-EDS/02-Python/Satellite-Person-Names.csv
################################
Vacuum Databases
### Done!! ############################################
